In [ ]:
import requests
import base64
import datetime
from urllib.parse import urlencode
from pandas import DataFrame
import pandas as pd
import requests
import numpy as np

In [ ]:
# Preencher com client_id gerado através do link developer.spotify.com/dashboard/
client_id = '' 
client_secret = ''

In [ ]:
def gerar_client_credentials_base64(client_id, client_secret):
    client_creds = f"{client_id}:{client_secret}"
    client_creds_base64 = base64.b64encode(client_creds.encode())
    
    return client_creds_base64.decode()

In [ ]:
class SpotifyAPI(object):
    access_token = ''
    expires_in = ''
    client_id = ''
    client_secret = ''
    url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret):
        super().__init__()
        self.client_id = client_id
        self.client_secret = client_secret
        
    def autenticacao(self):
        r = requests.post(
            self.url, 
            data={
                "grant_type": "client_credentials"
            }, 
            headers={
                "Authorization": f"Basic {gerar_client_credentials_base64(self.client_id, self.client_secret)}"
            })

        if r.status_code == 200:
            resposta = r.json()
            
            self.access_token = resposta['access_token']
            
            return True
        return False

In [ ]:
spotify = SpotifyAPI(client_id, client_secret)
spotify.autenticacao()

## Buscando atributo de popularidade

In [ ]:
features_url = "https://api.spotify.com/v1/tracks?ids="

In [ ]:
df = pd.read_csv('tracks_features.csv')
df = df[(df['year'] >= 1922) & (df['year'] < 1980)]

In [ ]:
def obterAtributos(track_id) :
    try:
        headers= { "Authorization": "Bearer {}".format(spotify.access_token) }
        
        features_request = requests.get(features_url + track_id, headers=headers)
        
        if (features_request.status_code == 401):
            spotify.autenticacao()
            
            headers= { "Authorization": "Bearer {}".format(spotify.access_token) }
            
            features_request = requests.get(features_url + track_id, headers=headers)
            
        features = features_request.json()

        if(features is not None):
            return [{
                'id': o['id'],
                'popularity': o['popularity']
            } for o in features['tracks']]
        else:
            print('zero')
            return []
    except:
        print('erro')
        return []

In [ ]:
def obter_popularidade(spotify_id, popularidades):
    try:
        return [popularidade for popularidade in popularidades 
                   if popularidade['id'] == spotify_id][0]['popularity']
    except:
        return -1

In [ ]:
df_popularidade = pd.DataFrame()

for index, qte in enumerate(np.arange(0, len(df), 50, dtype=int)):
    linhas = df[qte:qte+49]

    popularidades = obterAtributos(','.join(linhas['id'].tolist()))

    for index, linha in linhas.iterrows():
        linha = linha.copy()

        linhas.loc[index, 'popularity'] = obter_popularidade(linha['id'], popularidades)
    
    df_popularidade = pd.concat([df_popularidade, linhas])
    print(len(df_popularidade))
    linhas = pd.DataFrame()

In [ ]:
df_popularidade.to_excel('tracks_features_plus_popularity_1922_1979.xlsx')